In [1]:
import sys, json
import pandas as pd
from pathlib import Path
from IPython.display import HTML, display
from decouple import config
sys.path.append("../src/")

from openai import OpenAI
from decouple import config
openai_client = OpenAI(api_key=config("OPENAI_API_KEY"))

from llm_helpers import chatgpt_ask

pd.set_option('display.max_columns', 1000, 'display.width', 1000, 'display.max_rows',1000)

data_dir = Path(".").absolute().parent/"data"
ls = lambda p:print("\n".join(map(str,p.iterdir())))

ls(data_dir)

c:\Users\Elia Yakin\Documents\llm_workshop\data\sample_apps.parquet


In [2]:
df = pd.read_parquet(data_dir / "sample_apps.parquet").sample(9)
df

,bundle_id,title,description,store_url,category_names,ios
49136,com.tripledot.woodoku,Woodoku - Block Puzzle Games,Woodoku: a wood block puzzle game meets a sudo...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False
39809,com.playrix.fishdomdd.gplay,Fishdom,Never Fishdomed before? Take a deep breath and...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False
32145,com.macys.android,Macy's,The latest version of the Macy’s app is better...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False
19386,com.exoticmatch.game,Zen Match,Playing Zen Match for 10 minutes a day sharpen...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False
13106,com.breakingnewsbrief.app,US News - Breaking News Brief,Breaking News Brief delivers the world’s most ...,https://play.google.com/store/apps/details?id=...,"NEWS_AND_MAGAZINES,APPLICATION",False
15820,com.creditkarma.mobile,Credit Karma,• Check your free credit scores – Learn what a...,https://play.google.com/store/apps/details?id=...,"FINANCE,APPLICATION",False
5959,664575829,Fishdom,Never Fishdomed before? Then take a deep breat...,https://apps.apple.com/us/app/fishdom/id664575...,"Games,Puzzle,Entertainment,Simulation",True
54216,de.cellular.ottohybrid,OTTO – Shopping & Möbel,Install the OTTO app now 📲 and shop fashion tr...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False
62571,slots.pcg.casino.games.free.android,Cash Frenzy™ - Casino Slots,"⭐Keep spinning, keep fun going! ⭐ \n\nReady to...",https://play.google.com/store/apps/details?id=...,"GAME_CASINO,GAME",False


In [3]:
df.shape

(9, 6)

## Ask ChatGPT a question on every row

In [4]:
def chatgpt_ask(question: str, model_name="gpt-3.5-turbo") -> str:
    """Ask a question to chatgpt, apply it to every row of a pandas dataframe and return the answer"""
    def pandas_func(row)-> str:
        try:
            prompt = question.format(**(dict(row.items())))
            completion = openai_client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
            )
            
            ret = completion.choices[0].message.content.strip()

            return ret
        except Exception as e:
            print(e)
            return None
    return pandas_func


In [6]:
df["for_kids"] = df.apply(chatgpt_ask("Given the app description:\n{description}\n\nIs it for kids?\nAnswer only yes or no"),axis=1)
# df["for_kids"] = df.chatgpt_ask("Given the app description:\n{description}\n\nIs it for kids?\nAnswer only yes or no")
df


,bundle_id,title,description,store_url,category_names,ios,for_kids
49136,com.tripledot.woodoku,Woodoku - Block Puzzle Games,Woodoku: a wood block puzzle game meets a sudo...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,Yes
39809,com.playrix.fishdomdd.gplay,Fishdom,Never Fishdomed before? Take a deep breath and...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,Yes
32145,com.macys.android,Macy's,The latest version of the Macy’s app is better...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False,No.
19386,com.exoticmatch.game,Zen Match,Playing Zen Match for 10 minutes a day sharpen...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,Yes.
13106,com.breakingnewsbrief.app,US News - Breaking News Brief,Breaking News Brief delivers the world’s most ...,https://play.google.com/store/apps/details?id=...,"NEWS_AND_MAGAZINES,APPLICATION",False,No
15820,com.creditkarma.mobile,Credit Karma,• Check your free credit scores – Learn what a...,https://play.google.com/store/apps/details?id=...,"FINANCE,APPLICATION",False,No
5959,664575829,Fishdom,Never Fishdomed before? Then take a deep breat...,https://apps.apple.com/us/app/fishdom/id664575...,"Games,Puzzle,Entertainment,Simulation",True,Yes
54216,de.cellular.ottohybrid,OTTO – Shopping & Möbel,Install the OTTO app now 📲 and shop fashion tr...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False,No.
62571,slots.pcg.casino.games.free.android,Cash Frenzy™ - Casino Slots,"⭐Keep spinning, keep fun going! ⭐ \n\nReady to...",https://play.google.com/store/apps/details?id=...,"GAME_CASINO,GAME",False,No


## Question:
Please rewrite the code such that the response to a yes/no question would be boolean (`True` or `False`)

In [7]:
df['for_kids_bool'] = (df.for_kids=="Yes")
df

,bundle_id,title,description,store_url,category_names,ios,for_kids,for_kids_bool
49136,com.tripledot.woodoku,Woodoku - Block Puzzle Games,Woodoku: a wood block puzzle game meets a sudo...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,Yes,True
39809,com.playrix.fishdomdd.gplay,Fishdom,Never Fishdomed before? Take a deep breath and...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,Yes,True
32145,com.macys.android,Macy's,The latest version of the Macy’s app is better...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False,No.,False
19386,com.exoticmatch.game,Zen Match,Playing Zen Match for 10 minutes a day sharpen...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,Yes.,False
13106,com.breakingnewsbrief.app,US News - Breaking News Brief,Breaking News Brief delivers the world’s most ...,https://play.google.com/store/apps/details?id=...,"NEWS_AND_MAGAZINES,APPLICATION",False,No,False
15820,com.creditkarma.mobile,Credit Karma,• Check your free credit scores – Learn what a...,https://play.google.com/store/apps/details?id=...,"FINANCE,APPLICATION",False,No,False
5959,664575829,Fishdom,Never Fishdomed before? Then take a deep breat...,https://apps.apple.com/us/app/fishdom/id664575...,"Games,Puzzle,Entertainment,Simulation",True,Yes,True
54216,de.cellular.ottohybrid,OTTO – Shopping & Möbel,Install the OTTO app now 📲 and shop fashion tr...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False,No.,False
62571,slots.pcg.casino.games.free.android,Cash Frenzy™ - Casino Slots,"⭐Keep spinning, keep fun going! ⭐ \n\nReady to...",https://play.google.com/store/apps/details?id=...,"GAME_CASINO,GAME",False,No,False
